The imported dependencies are available from the lead contact upon request (lee.cronin@glasgow.ac.uk)

In [ ]:
from AnalyticalLabware import SpinsolveNMR
from commanduinolabware import CommanduinoLabware
from commanduino.commanddevices.commandlinearaccelstepper import CommandLinearAccelStepper

Initialisation of the NMR spectrometer and the peristaltic pump

In [ ]:
nmr = SpinsolveNMR()

config_file = "arduino_board_config.json"
board = CommanduinoLabware(config=config_file)
perry: CommandLinearAccelStepper = board.devices['peri']

Helper commands to find out which experimental protocols are available on the current Spinsolve spectrometer

In [ ]:
# nmr.protocols_list

In [ ]:
# nmr.cmd.get_protocol("SHIM 1H SAMPLE")

NMR convenience methods

In [ ]:
def quick_nmr():
    nmr.get_spectrum(('1D EXTENDED+',{
        'Number': '1',
        'AcquisitionTime': '3.2',
        'RepetitionTime': '4'
    }))

def test_nmr():
    nmr.sample = "test"
    # nmr.get_spectrum(('1D PROTON',{'Scan': 'QuickScan'}))
    quick_nmr()

def shim_on_sample(ref: float):
    og_sample = nmr.sample
    nmr.sample = "shim"
    nmr.get_spectrum(
        (
            'SHIM 1H SAMPLE',
            {
                'Mode': 'Manual',
                'manualStart': -25,
                'manualEnd': 35,
                'SampleReference': ref,
                'Shim': 'QuickShim1'
            }
        )
    )
    nmr.sample = og_sample

def lock_on_sample(ref: float):
    og_sample = nmr.sample
    nmr.sample = "lock"
    nmr.get_spectrum(
        (
            'SHIM 1H SAMPLE',
            {
                'Mode': 'Manual',
                'manualStart': -25,
                'manualEnd': 35,
                'SampleReference': ref,
                'Shim': 'LockAndCalibrateOnly'
            }
        )
    )
    nmr.sample = og_sample

def get_wet_sup():
    nmr.get_spectrum(
        (
            '1D WET SUP',
            {
                'Mode': 'Manual',
                'satFrequency1': 1.6,
                'CorrectionFactor': 1,
                'Dummy': 0,
                'Number': 2,
                'AcquisitionTime': 3.2,
                'RepetitionTime': 4
            }
        )
    )

Peristaltic pump convenience methods

In [ ]:
import threading

_stop_flag = threading.Event()

def run_pump():
    while not _stop_flag.is_set():
        # move in 0.1 mL increments
        perry.move(15625)

def start_pump():
    pump_thread = threading.Thread(target=run_pump)
    _stop_flag.clear()
    pump_thread.start()

def stop_pump():
    _stop_flag.set()

def move_volume(volume: float):
    for _ in range(round(volume)):
        perry.move(156250)
    perry.move(156250 * volume % 1)

Test NMR is calibrated correctly

In [ ]:
start_pump()

In [ ]:
test_nmr()

In [ ]:
lock_on_sample(1.6)

In [ ]:
stop_pump()

Define experimental parameters here

In [ ]:
nmr.sample = 'MGS263'
total_runtime = 4*60*60 # seconds

In [ ]:
from datetime import datetime, timedelta

time_elapsed = 0

start_time = datetime.now()
last_shim_time = datetime.now() # - timedelta(hours=1)
print(f"Starting sampling at {start_time}")

while time_elapsed < total_runtime:

    time_since_last_shim = (datetime.now() - last_shim_time).total_seconds()
    print(f"Time since last shim: {str(timedelta(seconds=time_since_last_shim))}")
    if time_since_last_shim > 15 * 60:
        print("Shim too old. Recalibrating...")
        shim_on_sample(1.6)
        last_shim_time = datetime.now()
    lock_on_sample(1.6)
    get_wet_sup()
    move_volume(1)

    time_elapsed = (datetime.now() - start_time).total_seconds()
    time_left = timedelta(seconds=total_runtime - time_elapsed)
    print(f"Time left: {str(time_left if time_left.total_seconds() > 0 else 0)}")

print("Experiment finished.")

Cleaning of the flow cell by reversing the pump direction

In [ ]:
# Invert pump direction to return sample
perry.reverted_direction = True
move_volume(5)
perry.reverted_direction = False